# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels]

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels]

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 122
intermediate_dim = 122
latent_dim = 32
batch_size = 1409
epochs = 5
hidden_layers = 8

class Train:
    def train():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu')(hidden_encoder)

        Train.mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)
        Train.logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            m_e, l_e = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = m_e + K.exp(l_e / 2) * epsilon
            return z

        z = Lambda(get_distrib)([Train.mean_encoder, Train.logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu")(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None)(hidden_decoder)

def vae_loss(x, x_decoded_mean):
    xent_loss = input_dim * keras.losses.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + Train.logvar_encoder - K.square(Train.mean_encoder) - K.exp(Train.logvar_encoder), axis=-1)
    return K.abs(K.mean(xent_loss + kl_loss))


In [4]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

features_arr = [2, 4,  8, 16, 32]
hidden_layers_arr = [2, 6, 10]

epoch_arr = [50]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = pd.DataFrame()

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h

    Train.train()

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
    vae_model = Model(inputs = Train.x, outputs = Train.x_ )
    vae_model.compile(optimizer = optimizer, loss = vae_loss, metrics = ['accuracy'] )

    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    vae_model.fit(x = x_train[:train_size,:], y = x_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  validation_data = (x_test, x_test),
                  verbose = 1)
    
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = x_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = x_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)


 
 Current Layer Attributes - epochs:50 hidden layers:2 features count:2
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 1s - loss: 25858278431901.1602 - acc: 0.0061 - val_loss: 7.8063 - val_acc: 0.0109
Epoch 2/50
112720/112720 [==============================] - 1s - loss: 3.1601 - acc: 0.0069 - val_loss: 6.9161 - val_acc: 0.0100
Epoch 3/50
112720/112720 [==============================] - 1s - loss: 3.2953 - acc: 0.0067 - val_loss: 6.6863 - val_acc: 0.0096
Epoch 4/50
112720/112720 [==============================] - 1s - loss: 2.8461 - acc: 0.0065 - val_loss: 5.9909 - val_acc: 0.0078
Epoch 5/50
112720/112720 [==============================] - 1s - loss: 2.9632 - acc: 0.0065 - val_loss: 6.0881 - val_acc: 0.0089
Epoch 6/50
112720/112720 [==============================] - 1s - loss: 2.8493 - acc: 0.0065 - val_loss: 6.1279 - val_acc: 0.0094
Epoch 7/50
112720/112720 [==============================] - 1s - loss: 2.3501 - acc: 0.00

112720/112720 [==============================] - 1s - loss: 6.0041 - acc: 0.0018 - val_loss: 154.0604 - val_acc: 0.0090
Epoch 11/50
112720/112720 [==============================] - 1s - loss: 5.9748 - acc: 0.0018 - val_loss: 148.9816 - val_acc: 0.0092
Epoch 12/50
112720/112720 [==============================] - 1s - loss: 6.1378 - acc: 0.0018 - val_loss: 148.2061 - val_acc: 0.0091
Epoch 13/50
112720/112720 [==============================] - 1s - loss: 6.7567 - acc: 0.0019 - val_loss: 144.9416 - val_acc: 0.0092
Epoch 14/50
112720/112720 [==============================] - 1s - loss: 6.0175 - acc: 0.0019 - val_loss: 143.1520 - val_acc: 0.0090
Epoch 15/50
112720/112720 [==============================] - 1s - loss: 6.2678 - acc: 0.0018 - val_loss: 139.9805 - val_acc: 0.0089
Epoch 16/50
112720/112720 [==============================] - 1s - loss: 6.2400 - acc: 0.0018 - val_loss: 138.2695 - val_acc: 0.0091
Epoch 17/50
112720/112720 [==============================] - 1s - loss: 6.2751 - acc: 0.

112720/112720 [==============================] - 0s - loss: 4.2585 - acc: 0.0049 - val_loss: 1605.3748 - val_acc: 0.0031
Epoch 21/50
112720/112720 [==============================] - 1s - loss: 4.3591 - acc: 0.0049 - val_loss: 1580.4515 - val_acc: 0.0032
Epoch 22/50
112720/112720 [==============================] - 1s - loss: 4.1467 - acc: 0.0049 - val_loss: 1555.7949 - val_acc: 0.0032
Epoch 23/50
112720/112720 [==============================] - 1s - loss: 4.4550 - acc: 0.0049 - val_loss: 1527.0437 - val_acc: 0.0033
Epoch 24/50
112720/112720 [==============================] - 1s - loss: 4.0495 - acc: 0.0049 - val_loss: 1500.7872 - val_acc: 0.0034
Epoch 25/50
112720/112720 [==============================] - 1s - loss: 4.5215 - acc: 0.0049 - val_loss: 1493.7885 - val_acc: 0.0033
Epoch 26/50
112720/112720 [==============================] - 1s - loss: 4.0239 - acc: 0.0049 - val_loss: 1491.7838 - val_acc: 0.0034
Epoch 27/50
112720/112720 [==============================] - 1s - loss: 4.3339 - 

112720/112720 [==============================] - 1s - loss: 126527214.5422 - acc: 0.0328 - val_loss: 162605150.5627 - val_acc: 0.0396
Epoch 27/50
112720/112720 [==============================] - 1s - loss: 126389388.4845 - acc: 0.0331 - val_loss: 161924478.1718 - val_acc: 0.0397
Epoch 28/50
112720/112720 [==============================] - 1s - loss: 126261066.7085 - acc: 0.0330 - val_loss: 158296074.5017 - val_acc: 0.0396
Epoch 29/50
112720/112720 [==============================] - 1s - loss: 126132795.3147 - acc: 0.0332 - val_loss: 156276100.4415 - val_acc: 0.0397
Epoch 30/50
112720/112720 [==============================] - 1s - loss: 126013507.8034 - acc: 0.0332 - val_loss: 154699879.8382 - val_acc: 0.0398
Epoch 31/50
112720/112720 [==============================] - 1s - loss: 125899154.2454 - acc: 0.0330 - val_loss: 152655537.6534 - val_acc: 0.0399
Epoch 32/50
112720/112720 [==============================] - 1s - loss: 125796847.7556 - acc: 0.0328 - val_loss: 150702522.2640 - val_ac

112720/112720 [==============================] - 1s - loss: 304803.2948 - acc: 0.0058 - val_loss: 16441033.9308 - val_acc: 0.0016
Epoch 31/50
112720/112720 [==============================] - 1s - loss: 299770.2349 - acc: 0.0057 - val_loss: 16109049.0163 - val_acc: 0.0016
Epoch 32/50
112720/112720 [==============================] - 1s - loss: 295001.0043 - acc: 0.0058 - val_loss: 15832520.6417 - val_acc: 0.0016
Epoch 33/50
112720/112720 [==============================] - 1s - loss: 290533.6061 - acc: 0.0058 - val_loss: 15432839.2347 - val_acc: 0.0016
Epoch 34/50
112720/112720 [==============================] - 1s - loss: 285639.9410 - acc: 0.0058 - val_loss: 15166270.3033 - val_acc: 0.0016
Epoch 35/50
112720/112720 [==============================] - 1s - loss: 281223.8917 - acc: 0.0058 - val_loss: 14852928.6724 - val_acc: 0.0016
Epoch 36/50
112720/112720 [==============================] - 1s - loss: 276978.9954 - acc: 0.0058 - val_loss: 14553418.6116 - val_acc: 0.0016
Epoch 37/50
112720

...

112720/112720 [==============================] - 2s - loss: 2.2663 - acc: 0.0044 - val_loss: 16.2838 - val_acc: 0.0114
Epoch 13/50
112720/112720 [==============================] - 2s - loss: 2.2100 - acc: 0.0041 - val_loss: 16.3530 - val_acc: 0.0113
Epoch 14/50
112720/112720 [==============================] - 2s - loss: 1.9781 - acc: 0.0040 - val_loss: 16.2449 - val_acc: 0.0112
Epoch 15/50
112720/112720 [==============================] - 2s - loss: 2.1380 - acc: 0.0041 - val_loss: 16.2878 - val_acc: 0.0111
Epoch 16/50
112720/112720 [==============================] - 2s - loss: 2.0578 - acc: 0.0039 - val_loss: 16.1554 - val_acc: 0.0110
Epoch 17/50
112720/112720 [==============================] - 2s - loss: 2.2773 - acc: 0.0039 - val_loss: 16.2945 - val_acc: 0.0109
Epoch 18/50
112720/112720 [==============================] - 2s - loss: 2.1112 - acc: 0.0038 - val_loss: 16.2549 - val_acc: 0.0108
Epoch 19/50
112720/112720 [==============================] - 2s - loss: 2.2830 - acc: 0.0039 - 

112720/112720 [==============================] - 3s - loss: 2.7026 - acc: 0.0026 - val_loss: 4.4676 - val_acc: 7.9844e-04
Epoch 23/50
112720/112720 [==============================] - 3s - loss: 2.3954 - acc: 0.0026 - val_loss: 4.4979 - val_acc: 7.9844e-04
Epoch 24/50
112720/112720 [==============================] - 3s - loss: 2.7072 - acc: 0.0026 - val_loss: 4.5249 - val_acc: 7.9844e-04
Epoch 25/50
112720/112720 [==============================] - 3s - loss: 2.6389 - acc: 0.0026 - val_loss: 4.5545 - val_acc: 7.9844e-04
Epoch 26/50
112720/112720 [==============================] - 3s - loss: 2.7775 - acc: 0.0026 - val_loss: 4.5916 - val_acc: 7.9844e-04
Epoch 27/50
112720/112720 [==============================] - 3s - loss: 2.8940 - acc: 0.0026 - val_loss: 4.6317 - val_acc: 7.9844e-04
Epoch 28/50
112720/112720 [==============================] - 3s - loss: 2.4215 - acc: 0.0026 - val_loss: 4.7347 - val_acc: 7.9844e-04
Epoch 29/50
112720/112720 [==============================] - 3s - loss: 2.

112720/112720 [==============================] - 3s - loss: 2.4242 - acc: 0.0055 - val_loss: 8.4037 - val_acc: 0.0018
Epoch 33/50
112720/112720 [==============================] - 3s - loss: 2.4745 - acc: 0.0055 - val_loss: 8.2708 - val_acc: 0.0018
Epoch 34/50
112720/112720 [==============================] - 3s - loss: 2.7765 - acc: 0.0055 - val_loss: 8.3727 - val_acc: 0.0018
Epoch 35/50
112720/112720 [==============================] - 3s - loss: 2.9161 - acc: 0.0055 - val_loss: 8.4060 - val_acc: 0.0018
Epoch 36/50
112720/112720 [==============================] - 3s - loss: 2.9246 - acc: 0.0055 - val_loss: 8.1542 - val_acc: 0.0018
Epoch 37/50
112720/112720 [==============================] - 3s - loss: 2.7420 - acc: 0.0055 - val_loss: 8.4035 - val_acc: 0.0018
Epoch 38/50
112720/112720 [==============================] - 3s - loss: 2.4429 - acc: 0.0055 - val_loss: 8.4214 - val_acc: 0.0018
Epoch 39/50
112720/112720 [==============================] - 3s - loss: 2.6584 - acc: 0.0055 - val_los

112720/112720 [==============================] - 3s - loss: 2.1348 - acc: 4.2583e-04 - val_loss: 9.2473 - val_acc: 0.0042
Epoch 43/50
112720/112720 [==============================] - 3s - loss: 1.9647 - acc: 4.2583e-04 - val_loss: 9.1289 - val_acc: 0.0042
Epoch 44/50
112720/112720 [==============================] - 3s - loss: 2.2190 - acc: 4.2583e-04 - val_loss: 9.1283 - val_acc: 0.0042
Epoch 45/50
112720/112720 [==============================] - 3s - loss: 2.3317 - acc: 4.2583e-04 - val_loss: 9.1243 - val_acc: 0.0042
Epoch 46/50
112720/112720 [==============================] - 3s - loss: 2.2283 - acc: 4.2583e-04 - val_loss: 9.1263 - val_acc: 0.0042
Epoch 47/50
112720/112720 [==============================] - 3s - loss: 2.4206 - acc: 4.2583e-04 - val_loss: 9.0753 - val_acc: 0.0042
Epoch 48/50
112720/112720 [==============================] - 3s - loss: 2.5235 - acc: 4.2583e-04 - val_loss: 9.1002 - val_acc: 0.0042
Epoch 49/50
112720/112720 [==============================] - 3s - loss: 2.

112720/112720 [==============================] - 2s - loss: 2.5510 - acc: 6.2988e-04 - val_loss: 20.1741 - val_acc: 5.7665e-04
Epoch 50/50
19726/22544 [=========================>....] - ETA: 0s
 Train Acc: 0.00044357700971886516, Test Acc: 0.0005766501126345247
 
 Current Layer Attributes - epochs:50 hidden layers:10 features count:32
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 4s - loss: 2.7738 - acc: 0.0037 - val_loss: 28.9895 - val_acc: 0.0063
Epoch 2/50
112720/112720 [==============================] - 2s - loss: 2.7571 - acc: 0.0044 - val_loss: 29.4855 - val_acc: 0.0063
Epoch 3/50
112720/112720 [==============================] - 2s - loss: 2.7743 - acc: 0.0044 - val_loss: 29.5967 - val_acc: 0.0062
Epoch 4/50
112720/112720 [==============================] - 2s - loss: 3.1826 - acc: 0.0044 - val_loss: 29.7883 - val_acc: 0.0063
Epoch 5/50
112720/112720 [==============================] - 2s - loss: 2.3408 - acc: 0.0044 

In [5]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
3,50,16,2,0.034422,0.039257
9,50,32,6,0.003903,0.010424
0,50,2,2,0.005500,0.008694
1,50,4,2,0.001331,0.008206
14,50,32,10,0.003992,0.006254
12,50,8,10,0.000444,0.004214
2,50,8,2,0.003992,0.003327
11,50,4,10,0.004791,0.001774
4,50,32,2,0.005057,0.001597
8,50,16,6,0.004081,0.001198


In [6]:
scores.to_pickle("dataset/vae_only_feature_extraction_scores.pkl")